In [1]:
import tensorflow as tf
from tensorflow.contrib import rnn


In [2]:
import pandas as pd

In [3]:
data=pd.read_csv("allAtt_onehot_large_train.csv")
dataT=pd.read_csv("allAtt_onehot_large_test.csv")
print(data.head())
print(data.shape)


   FTR  HTGS  ATGS  HTGC  ATGC  HTP  ATP  HM1  HM2  HM3   ...    ATWinStreak5  \
0    0   0.0   0.0   0.0   0.0  0.0  0.0    2    2    2   ...               0   
1    1   0.0   0.0   0.0   0.0  0.0  0.0    2    2    2   ...               0   
2    1   0.0   0.0   0.0   0.0  0.0  0.0    2    2    2   ...               0   
3    1   0.0   0.0   0.0   0.0  0.0  0.0    2    2    2   ...               0   
4    1   0.0   0.0   0.0   0.0  0.0  0.0    2    2    2   ...               0   

   ATLossStreak3  ATLossStreak5  HTGD  ATGD  DiffPts  DiffFormPts  DiffLP  \
0              0              0   0.0   0.0      0.0          0.0   -15.0   
1              0              0   0.0   0.0      0.0          0.0    10.0   
2              0              0   0.0   0.0      0.0          0.0   -16.0   
3              0              0   0.0   0.0      0.0          0.0    -1.0   
4              0              0   0.0   0.0      0.0          0.0     5.0   

   final1  final2  
0     1.0     0.0  
1     0.0 

In [4]:

hm_epochs=10
n_classes = 2
batch_size = 1
chunk_size=27
n_chunks=1
rnn_size=64


x = tf.placeholder('float', [None, n_chunks,chunk_size])
y = tf.placeholder('float')

In [5]:
def recurrent_neural_model(x):
    layer = {'weights':tf.Variable(tf.random_normal([rnn_size,n_classes])),
            'biases':tf.Variable(tf.random_normal([n_classes]))}
    
    x=tf.transpose(x,[1,0,2])
    print("transpose",x)
    x=tf.reshape(x,[-1,chunk_size])
    print("reshape",x)
    x=tf.split(x,n_chunks)
    print("split",x)
    lstm_cell = rnn.BasicLSTMCell(rnn_size)
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)
    


    output = tf.matmul(outputs[-1],layer['weights']) + layer['biases']

    return output

In [6]:
def train_neural_network(x):
    prediction = recurrent_neural_model(x)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y) )
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        hm_epochs=10
        for epoch in range(hm_epochs):
            epoch_loss = 0
            for i in range(0,data.shape[0],batch_size):
                epoch_x, epoch_y = data.iloc[i:i+batch_size,1:28].values,data.iloc[i:i+batch_size,28:].values
                epoch_x=epoch_x.reshape((batch_size,n_chunks,chunk_size))
                _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
                epoch_loss += c

            print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)

        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))

        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy Train:',accuracy.eval({x:data.iloc[:,1:28].values.reshape((-1,n_chunks,chunk_size)), 
                                               y:data.iloc[:,28:].values}))
        print('Accuracy Test:',accuracy.eval({x:dataT.iloc[:,1:28].values.reshape((-1,n_chunks,chunk_size)), 
                                              y:dataT.iloc[:,28:].values}))

In [7]:
train_neural_network(x)

transpose Tensor("transpose:0", shape=(1, ?, 27), dtype=float32)
reshape Tensor("Reshape:0", shape=(?, 27), dtype=float32)
split [<tf.Tensor 'split:0' shape=(?, 27) dtype=float32>]
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Epoch 0 completed out of 10 loss: 1216.3304373975843
Epoch 1 completed out of 10 loss: 1075.46176257357
Epoch 2 completed out of 10 loss: 980.8723999504
Epoch 3 completed out of 10 loss: 864.5714094936848
Epoch 4 completed out of 10 loss: 738.5878906969447
Epoch 5 completed out of 10 loss: 614.039942086878
Epoch 6 completed out of 10 loss: 492.0997193742478
Epoch 7 completed out of 10 loss: 377.4936973380577
Epoch 8 completed out of 10 loss: 277.3404195612128
Epoch 9 completed out of 10 loss: 196.47607771527558
Accuracy Train: 0.97849464
Accuracy Test: 0.80875
